In [ ]:
from google.colab import drive
drive.mount("/content/drive")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np


import os
import re
import string
from tqdm import tqdm

import numpy as np
import pandas as pd

import torch

from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
df_big = pd.read_csv("/content/drive/MyDrive/University/wiki_movie_plots_deduped.csv")
df_top_100 = pd.read_csv("/content/drive/MyDrive/University/movies.csv")

In [ ]:
print(df_big.shape)
df_big.head()

(34886, 8)


,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


In [ ]:
print(df_top_100.shape)
df_top_100.head()

(100, 5)


,rank,title,genre,wiki_plot,imdb_plot
0,0,The Godfather,"[u' Crime', u' Drama']","On the day of his only daughter's wedding, Vit...","In late summer 1945, guests are gathered for t..."
1,1,The Shawshank Redemption,"[u' Crime', u' Drama']","In 1947, banker Andy Dufresne is convicted of ...","In 1947, Andy Dufresne (Tim Robbins), a banker..."
2,2,Schindler's List,"[u' Biography', u' Drama', u' History']","In 1939, the Germans move Polish Jews into the...",The relocation of Polish Jews from surrounding...
3,3,Raging Bull,"[u' Biography', u' Drama', u' Sport']","In a brief scene in 1964, an aging, overweight...","The film opens in 1964, where an older and fat..."
4,4,Casablanca,"[u' Drama', u' Romance', u' War']",It is early December 1941. American expatriate...,"In the early years of World War II, December 1..."


In [ ]:
df = df_top_100

In [ ]:
# regex-pattern to remove whitespaces


def clean_string(text: str) -> str:
    """
    function that cleans string
    
    deletes punctuations, emoji, leaves 1 whitespace, makes everything lowercase, strips
    
    """
    re_whitespace = re.compile(r"(\s(?=\s{1,}))")
    
    text = text.translate(str.maketrans(string.punctuation, " " * len(string.punctuation))).lower()
    text = re_whitespace.sub(r"", text)

    return text.strip()



def find_sim(df, plot_col, title_col, max_features = 10000, threshold = 0.5):

    plot = df[plot_col].apply(lambda x: clean_string(x))

    tfidf_vec = TfidfVectorizer(stop_words='english', 
                            binary=True,
                            max_features=max_features)
    
    text_embeddings = tfidf_vec.fit_transform(plot).toarray().astype(np.float32)
                  
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    text_tensor = torch.from_numpy(text_embeddings).to(device)

    out_preds = []
    chunk = 128

    for i in tqdm(list(range(0, df.shape[0], chunk)) + [df.shape[0] - chunk]):
        
        # cosine similarity 
        arr = text_tensor[i : i + chunk] @ text_tensor.T

        indices = torch.nonzero(arr > threshold)

        preds = dict()
        for k in range(arr.shape[0]):
            preds[k] = []

        for ind in range(indices.size(0)):
            preds[indices[ind, 0].item()].append(indices[ind, 1].item())

        out_preds.extend(
            [
                (df.iloc[k + i].title, df.iloc[v].title.tolist())
                for k, v in preds.items()
            ]
        )
    
    out_preds = out_preds[: df.shape[0]]
    df_pred = pd.DataFrame(out_preds, columns=["index", "pred"])
    #df.set_index("index")

    return df_pred

In [ ]:

df_pred = find_sim(df_top_100, "wiki_plot", "title", max_features = 5000, threshold = 0.1)


100%|██████████| 2/2 [00:00<00:00, 18.82it/s]


In [ ]:
pd.set_option('display.max_colwidth', None)
df_pred

,index,pred
0,The Godfather,"[The Godfather, The Godfather: Part II]"
1,The Shawshank Redemption,"[The Shawshank Redemption, The Pianist]"
2,Schindler's List,"[Schindler's List, The Pianist, All Quiet on the Western Front]"
3,Raging Bull,"[Raging Bull, Goodfellas, The Graduate]"
4,Casablanca,"[Casablanca, The Godfather: Part II, West Side Story, From Here to Eternity, The Pianist]"
...,...,...
95,Rebel Without a Cause,"[12 Angry Men, Fargo, Double Indemnity, Rebel Without a Cause]"
96,Rear Window,"[The Apartment, The Deer Hunter, Nashville, Taxi Driver, Double Indemnity, Rear Window]"
97,The Third Man,"[Chinatown, Gladiator, Doctor Zhivago, The Deer Hunter, The French Connection, The Maltese Falcon, Double Indemnity, The Third Man]"
98,North by Northwest,"[The Maltese Falcon, North by Northwest]"
